# Notebook 7: Script for a new ecosystem
### iGED : Global Systems Dynamics Initiative

This script is aimed to be a user-friendly code where to introduce the data from  a new ecosystem and incorporate it into the DataFrames studied by Global Systems Dynamics Initiative at MIT (iGED). 

As of (07/28/2020) the following cities have been included in our data:

  *  Ciudad Autónoma de Buenos Aires (CABA)
  *  Ciudad de México (CDMX)
  *  Santiago de Chile (SCL)
  *  Montevideo (MTV)
  *  Madrid (MAD)
  *  Sao Paulo (SAO)
  *  Aguascalientes (AGS)
  *  Guadalajara (GDL)
  *  Pachuca (HGO)
  *  Oaxaca (OAX)
  *  Valencia (VAL)
  
The final product of this code is a tidy DataFrame containing a description of 16 graph metrics associated to each of our ecosystems, together with a collaborativity metric. 

### Part 1: Packages

Installation of all the packages used to perform the analysis in this file

In [1]:
import pandas as pd
import numpy as np 
import networkx as nx
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import math
import numpy.linalg

### Part 2: Importing Data from the new city and construction of its Network

We import, in CSV format, the data from the new city's ecoystem, both in terms of its nodes and in terms of its edges. From this information we can build the corresponding network, and then evaluate it according to the studied *graph metrics*.

In [10]:
new_eco_name =  'Valencia' 
country =       'España'
new_eco_nd =    pd.read_csv('Ecosystems_from_GED/Gephi_stats/Gephi '+new_eco_name+' Stats.csv' )
new_eco_ed =    pd.read_csv('Ecosystems_from_GED/Gephi_edges/Gephi '+new_eco_name+' Edges.csv')

In [11]:
def build_graph(nodes, edges, rol_str, weight_str):
    '''
    Function which, from a list of nodes and edges, builds a NetworkX graph.
    
    In:
    - nodes       node list
    - edges       edge list
    - rol_str     name for the parameter that describes actor's role 
    - weight_str  name for the parameter that describes edge's weight
    
    Out
    A NetworkX DiGraph: G.
    '''
    #create DiGraph from edgelist
    G = nx.from_pandas_edgelist(edges,'Source','Target',\
        edge_attr=["Weight"],create_using=nx.DiGraph())
    
    
    #create (node ->role(node)) dictionary 
    rol = {nid: nodes[nodes['Id']==nid][rol_str].values[0] for nid in nodes['Id']}
    nx.set_node_attributes(G, rol, 'rol')
    
    #create (node -> weight(node)) dictionart 
    weight = {nid: nodes[nodes['Id']==nid][weight_str].values[0] for nid in nodes['Id']}
    nx.set_node_attributes(G,weight,'weight')
    
    
    G.remove_edges_from(nx.selfloop_edges(G))
    return G

Building of a NetworkX graph based on the obtained CSV data throughthe function `build_graph`:

In [12]:
#--------------------------------------------------------------
# Build NetworkX graph based on CSV info
#--------------------------------------------------------------
#     IMPORTANT: rol_str or weight_str might need to be 
#     updated according to the city's node stats CSV


new_eco_G = build_graph(new_eco_nd, new_eco_ed,'type','weight')

In [13]:
#----------------------------
# Global Metrics
#----------------------------
#     We obtain a pandas.core.series.Series by 
#     getting averages per column of the stats CSV.

averages = new_eco_nd.mean()
#averages
#averages.loc['avg strength']

### Part 3: Incorporating the new ecosystem to the existing Database

So far we have a new ecosystem, built from a Gephi file and the two corresponding node and edge CSV files. From this information we can measure the graph of the new ecosytem through several graph metrics, and then incorporate this data to the DataFrame describing all previously studied ecosystems. 

In [23]:
#----------------------------
# Tidy DataFrame Acess
#----------------------------
#     Access to the Tidy DataFrame containing information 
#     from previously studied cities.

df_concat = pd.read_csv('Referenced_CSVs/Tidy_DataFrame.csv')
df_concat = df_concat[['Ciudad', 'País', 'avg strength', 'weight', 'Degree', 
                       'Weighted Degree', 'Excentricidad', 'Clustering', 
                       'Diámetro', 'Radio', 'Camino más corto promedio', 
                       'Transitividad', 'Eficiencia Global', 'Small Worldness', 
                       'Rich Club Coefficient', 'Core Ratio', 'Central Point Dominance',
                       'Spectral radius', 'Modularidad', 'Average Collaborations',
                       'Colab 0', 'Colab 10', 'Colab 10 rescaled']]
df_concat

,Ciudad,País,avg strength,weight,Degree,Weighted Degree,Excentricidad,Clustering,Diámetro,Radio,Camino más corto promedio,Transitividad,Eficiencia Global,Small Worldness,Rich Club Coefficient,Core Ratio,Central Point Dominance,Spectral radius,Modularidad,Average Collaborations,Colab 0,Colab 10,Colab 10 rescaled
0,CABA,Argentina,3.244633,1.481579,4.385965,14.017544,4.798246,0.300215,6,3,3.355669,0.107392,0.325773,1.006481,0.866667,0.495614,0.251,1.868,0.527,13.48,5.773443,0.492850,5.928502
1,CDMX,México,3.477388,1.388629,3.658863,12.963211,5.622074,0.178788,7,4,3.820318,0.050000,0.289320,0.614973,0.361111,0.421405,0.202,1.910,0.623,12.33,3.636191,0.397667,4.976668
2,Santiago,Chile,3.417211,1.462564,3.948718,13.764103,5.041026,0.311157,6,3,3.227544,0.100744,0.338889,1.051860,1.000000,0.430769,0.525,1.890,0.535,13.04,5.571009,0.480988,5.809878
3,Montevideo,Uruguay,3.227742,2.749495,7.747475,25.939394,4.479798,0.367266,6,3,3.081116,0.224523,0.360837,1.197588,1.000000,0.565657,0.179,1.778,0.356,13.40,8.471255,0.588490,6.884901
4,Madrid,España,3.518387,1.362343,3.790795,12.861925,6.079498,0.266684,8,5,3.783517,0.081917,0.294653,0.930159,1.000000,0.439331,0.184,1.920,0.606,12.13,4.256043,0.426240,5.262402
5,Sao Paulo,Brasil,3.430669,1.253704,3.370370,11.694444,6.726852,0.295414,8,4,4.324031,0.078571,0.266719,1.071208,0.222222,0.421296,0.237,1.983,0.682,10.38,3.258518,0.384944,4.849443
6,AGS,México,3.631461,1.797917,4.854167,17.020833,4.510417,0.328739,5,3,3.120614,0.140110,0.363556,1.182042,1.000000,0.593750,0.314,1.868,0.523,8.21,3.660575,0.461698,5.616984
7,GDL,México,3.573935,1.855615,5.069519,17.668449,4.705882,0.313413,6,3,3.208499,0.124392,0.342444,1.009314,1.000000,0.540107,0.287,1.804,0.511,10.81,4.845969,0.477331,5.773305
8,Pachuca,México,3.570259,1.563200,4.064000,13.952000,4.496000,0.214898,6,3,3.162323,0.095514,0.351108,0.852531,0.500000,0.384000,0.401,1.860,0.525,9.52,3.686997,0.439707,5.397074
9,Oaxaca,México,3.906913,1.883221,4.375839,17.369128,4.456376,0.234400,5,3,3.320697,0.107699,0.333850,1.157058,1.000000,0.516779,0.435,1.846,0.533,6.39,2.495120,0.399422,4.994219


In [24]:
#df_concat.isnull().sum()
#df_concat.columns

In [25]:
#------------------------------------
# Formulas for Global graph metrics
#------------------------------------

def small_worldness(U):
    return nx.algorithms.smallworld.sigma(U,niter=1,nrand=2)

def central_point_dominance(G):
    betwennesses = nx.betweenness_centrality(G)
    b_max = max(betwennesses.values())
    N = len(betwennesses.keys())
    count = 0
    for i, b_i in betwennesses.items():
        count += ( b_max - b_i )/(N-1)
    return count

def spectral_radius(G):
    '''Formula to compute spectral radius rho of a graph. As rho is smaller, 
    Pr[virus infection] grows, and it's easier for an idea to spread 
    in an innovative network'''
    L = nx.normalized_laplacian_matrix(G)
    e = numpy.linalg.eigvals(L.A)
    e_abs = [abs(x) for x in e]
    return max(e_abs)

def core_ratio(G):
    return len(nx.k_core(G,k=2).nodes())/len(G.nodes())

def rich_club_coeffs(G):
    t_ok = True
    t = 0
    d = dict()
    while t_ok:
        nodes_large_degree=[]
        for x in G.nodes():
            if G.degree(x)>t:
                nodes_large_degree.append(x)
        core = G.subgraph(nodes_large_degree)
        edges_core = len(core.edges())
        nodes_core = len(core.nodes())
        if nodes_core<=1:
            t_ok = False
            break
        d[t] = (2*edges_core)/(nodes_core*(nodes_core-1))
        t += 1 
    return d        

def max_rich_club(G):
    rich_club=rich_club_coeffs(G)
    max_i=0
    
    for i in range(len(rich_club)):
        if rich_club[i]>rich_club[max_i]:
            max_i=i
    return rich_club[max_i]

def modularity(G):
    return nx.algorithms.community.quality.performance(G,nx.algorithms.community.modularity_max.greedy_modularity_communities(G))

def clustering_gephi(G):
    '''
    Inputs:
    - G an undirected graph
    Outputs
    The clustering coefficient of G. Note that this value s different from the one in NetworkX;
    '''
    total = 0
    count = 0
    for v in G.nodes():
        if nx.degree(G,v) >= 2:
            total += nx.clustering(G,v)
            count += 1
    return total/count

def colaborativity_formula_0_original(df):
    '''
    Original Collaborativity Formula during summer 2020 for project GED.
    Computes the colaborativity of a network G based on its measures, as shown below. 
    
    Input:
    - df : dataframe mapping each city to its graph metrics
    Returns:
    - Value describing the collaborativity of a graph, or network
    '''
    Avg_colabs =df['Average Collaborations']
    Clust = df['Clustering']
    Mod = df['Modularidad']
    return (1/2) * Avg_colabs * (Clust + (np.log10(Mod**2)*-1))

def colaborativity_formula_10_decided(df):
    Avg_colabs = df['Average Collaborations']  #1 quantity  
    efi = df['Eficiencia Global']              #2 quality   - comunication 
    tran = df['Transitividad']                 #            - robustness
    exc = df['Excentricidad']                  #            - preparation for future
    return 0.5*(np.log(1+(Avg_colabs/25))) + (efi * tran * np.sin(math.pi/exc) )**(1/3)

def colaborativity_formula_10_rescaled(df):
    return 1+10*colaborativity_formula_10_decided(df)


In [26]:
pd.set_option('max_columns', None)
df_concat

,Ciudad,País,avg strength,weight,Degree,Weighted Degree,Excentricidad,Clustering,Diámetro,Radio,Camino más corto promedio,Transitividad,Eficiencia Global,Small Worldness,Rich Club Coefficient,Core Ratio,Central Point Dominance,Spectral radius,Modularidad,Average Collaborations,Colab 0,Colab 10,Colab 10 rescaled
0,CABA,Argentina,3.244633,1.481579,4.385965,14.017544,4.798246,0.300215,6,3,3.355669,0.107392,0.325773,1.006481,0.866667,0.495614,0.251,1.868,0.527,13.48,5.773443,0.492850,5.928502
1,CDMX,México,3.477388,1.388629,3.658863,12.963211,5.622074,0.178788,7,4,3.820318,0.050000,0.289320,0.614973,0.361111,0.421405,0.202,1.910,0.623,12.33,3.636191,0.397667,4.976668
2,Santiago,Chile,3.417211,1.462564,3.948718,13.764103,5.041026,0.311157,6,3,3.227544,0.100744,0.338889,1.051860,1.000000,0.430769,0.525,1.890,0.535,13.04,5.571009,0.480988,5.809878
3,Montevideo,Uruguay,3.227742,2.749495,7.747475,25.939394,4.479798,0.367266,6,3,3.081116,0.224523,0.360837,1.197588,1.000000,0.565657,0.179,1.778,0.356,13.40,8.471255,0.588490,6.884901
4,Madrid,España,3.518387,1.362343,3.790795,12.861925,6.079498,0.266684,8,5,3.783517,0.081917,0.294653,0.930159,1.000000,0.439331,0.184,1.920,0.606,12.13,4.256043,0.426240,5.262402
5,Sao Paulo,Brasil,3.430669,1.253704,3.370370,11.694444,6.726852,0.295414,8,4,4.324031,0.078571,0.266719,1.071208,0.222222,0.421296,0.237,1.983,0.682,10.38,3.258518,0.384944,4.849443
6,AGS,México,3.631461,1.797917,4.854167,17.020833,4.510417,0.328739,5,3,3.120614,0.140110,0.363556,1.182042,1.000000,0.593750,0.314,1.868,0.523,8.21,3.660575,0.461698,5.616984
7,GDL,México,3.573935,1.855615,5.069519,17.668449,4.705882,0.313413,6,3,3.208499,0.124392,0.342444,1.009314,1.000000,0.540107,0.287,1.804,0.511,10.81,4.845969,0.477331,5.773305
8,Pachuca,México,3.570259,1.563200,4.064000,13.952000,4.496000,0.214898,6,3,3.162323,0.095514,0.351108,0.852531,0.500000,0.384000,0.401,1.860,0.525,9.52,3.686997,0.439707,5.397074
9,Oaxaca,México,3.906913,1.883221,4.375839,17.369128,4.456376,0.234400,5,3,3.320697,0.107699,0.333850,1.157058,1.000000,0.516779,0.435,1.846,0.533,6.39,2.495120,0.399422,4.994219


In [27]:
U = new_eco_G.to_undirected()

new_row = {'Ciudad': new_eco_name, 
           'País': country,
           'avg strength': averages.loc['avg strength'],
           'weight': averages.loc['weight'],
           'Degree': averages.loc['Degree'],
           'Weighted Degree': averages.loc['Weighted Degree'],
           'Excentricidad': averages.loc['Eccentricity'],
           'Clustering': clustering_gephi(U),
           'Diámetro': nx.diameter(U),
           'Radio': nx.radius(U),
           'Camino más corto promedio': nx.average_shortest_path_length(U),
           'Transitividad': nx.transitivity(U),
           'Eficiencia Global': nx.global_efficiency(U),
           'Small Worldness': small_worldness(U),
           'Rich Club Coefficient': max_rich_club(U),
           'Core Ratio': core_ratio(U),
           'Central Point Dominance': central_point_dominance(U),
           'Spectral radius': spectral_radius(U),
           'Modularidad': 0.37,
           'Average Collaborations': 15.05,
           'Colab 0': 0,
           'Colab 10': 0,
           'Colab 10 rescaled': 0}

#note that modularity and avg collabs can be modified to the data obtained from the ecosystem. 

#append row to the dataframe
df_concat = df_concat.append(new_row, ignore_index=True)
df_concat


,Ciudad,País,avg strength,weight,Degree,Weighted Degree,Excentricidad,Clustering,Diámetro,Radio,Camino más corto promedio,Transitividad,Eficiencia Global,Small Worldness,Rich Club Coefficient,Core Ratio,Central Point Dominance,Spectral radius,Modularidad,Average Collaborations,Colab 0,Colab 10,Colab 10 rescaled
0,CABA,Argentina,3.244633,1.481579,4.385965,14.017544,4.798246,0.300215,6,3,3.355669,0.107392,0.325773,1.006481,0.866667,0.495614,0.25100,1.868000,0.527,13.48,5.773443,0.492850,5.928502
1,CDMX,México,3.477388,1.388629,3.658863,12.963211,5.622074,0.178788,7,4,3.820318,0.050000,0.289320,0.614973,0.361111,0.421405,0.20200,1.910000,0.623,12.33,3.636191,0.397667,4.976668
2,Santiago,Chile,3.417211,1.462564,3.948718,13.764103,5.041026,0.311157,6,3,3.227544,0.100744,0.338889,1.051860,1.000000,0.430769,0.52500,1.890000,0.535,13.04,5.571009,0.480988,5.809878
3,Montevideo,Uruguay,3.227742,2.749495,7.747475,25.939394,4.479798,0.367266,6,3,3.081116,0.224523,0.360837,1.197588,1.000000,0.565657,0.17900,1.778000,0.356,13.40,8.471255,0.588490,6.884901
4,Madrid,España,3.518387,1.362343,3.790795,12.861925,6.079498,0.266684,8,5,3.783517,0.081917,0.294653,0.930159,1.000000,0.439331,0.18400,1.920000,0.606,12.13,4.256043,0.426240,5.262402
5,Sao Paulo,Brasil,3.430669,1.253704,3.370370,11.694444,6.726852,0.295414,8,4,4.324031,0.078571,0.266719,1.071208,0.222222,0.421296,0.23700,1.983000,0.682,10.38,3.258518,0.384944,4.849443
6,AGS,México,3.631461,1.797917,4.854167,17.020833,4.510417,0.328739,5,3,3.120614,0.140110,0.363556,1.182042,1.000000,0.593750,0.31400,1.868000,0.523,8.21,3.660575,0.461698,5.616984
7,GDL,México,3.573935,1.855615,5.069519,17.668449,4.705882,0.313413,6,3,3.208499,0.124392,0.342444,1.009314,1.000000,0.540107,0.28700,1.804000,0.511,10.81,4.845969,0.477331,5.773305
8,Pachuca,México,3.570259,1.563200,4.064000,13.952000,4.496000,0.214898,6,3,3.162323,0.095514,0.351108,0.852531,0.500000,0.384000,0.40100,1.860000,0.525,9.52,3.686997,0.439707,5.397074
9,Oaxaca,México,3.906913,1.883221,4.375839,17.369128,4.456376,0.234400,5,3,3.320697,0.107699,0.333850,1.157058,1.000000,0.516779,0.43500,1.846000,0.533,6.39,2.495120,0.399422,4.994219


In [29]:
#df_concat.isnull().sum()

In [30]:
'''
In the case that a K- nearest neighbors imputer is needed 
in order to obtain Modularity, Avg Collabs


from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)

df_numbers = df_concat.copy()
df_numbers=df_numbers.drop('Ciudad',axis=1)
df_numbers=df_numbers.drop('País',axis=1)


var = imputer.fit_transform(df_numbers)
df_numbers = pd.DataFrame(var)


df_numbers.columns = ['avg strength', 'weight', 'Degree', 'Weighted Degree', 'Excentricidad',
                       'Clustering', 'Diámetro', 'Radio', 'Camino más corto promedio', 'Transitividad',
                       'Eficiencia Global', 'Small Worldness', 'Rich Club Coefficient', 'Core Ratio',
                       'Central Point Dominance', 'Spectral radius','Modularidad', 
                      'Average Collaborations', 'Colab 0', 'Colab 10', 'Colab 10 rescaled']

df_concat['Modularidad'] = df_numbers['Modularidad']
df_concat['Average Collaborations'] = df_numbers['Average Collaborations']
df_concat 

'''


"\nIn the case that a K- nearest neighbors imputer is needed \nin order to obtain Modularity, Avg Collabs\n\n\nfrom sklearn.impute import KNNImputer\nimputer = KNNImputer(n_neighbors=3)\n\ndf_numbers = df_concat.copy()\ndf_numbers=df_numbers.drop('Ciudad',axis=1)\ndf_numbers=df_numbers.drop('País',axis=1)\n\n\nvar = imputer.fit_transform(df_numbers)\ndf_numbers = pd.DataFrame(var)\n\n\ndf_numbers.columns = ['avg strength', 'weight', 'Degree', 'Weighted Degree', 'Excentricidad',\n                       'Clustering', 'Diámetro', 'Radio', 'Camino más corto promedio', 'Transitividad',\n                       'Eficiencia Global', 'Small Worldness', 'Rich Club Coefficient', 'Core Ratio',\n                       'Central Point Dominance', 'Spectral radius','Modularidad', \n                      'Average Collaborations', 'Colab 0', 'Colab 10', 'Colab 10 rescaled']\n\ndf_concat['Modularidad'] = df_numbers['Modularidad']\ndf_concat['Average Collaborations'] = df_numbers['Average Collaborations']

### Part 4: Computation of Collaborativity

All the graph metrics corresponding to the new ecosystem have been computed! The missing piece are the following three measurements:

*   `Colaborativity 0`: which was originally utilized to measure colaborativity
*   `Colaborativity 10`: according to the 10th proposed colaborativity formula 
*   `Colaborativity 10 rescale`: according to the rescaled version of the 10th proposed colaborativity formula 

Funally, we build a DataFrame focusing on the rankings induced by each of the Colaborativity metrics 0 and 10. 

In [31]:
#------------------------------
# Collaborativity computation
#------------------------------

df_concat['Colab 0'] = colaborativity_formula_0_original(df_concat)
df_concat['Colab 10'] = colaborativity_formula_10_decided(df_concat)
df_concat['Colab 10 rescaled'] = colaborativity_formula_10_rescaled(df_concat)
df_concat

,Ciudad,País,avg strength,weight,Degree,Weighted Degree,Excentricidad,Clustering,Diámetro,Radio,Camino más corto promedio,Transitividad,Eficiencia Global,Small Worldness,Rich Club Coefficient,Core Ratio,Central Point Dominance,Spectral radius,Modularidad,Average Collaborations,Colab 0,Colab 10,Colab 10 rescaled
0,CABA,Argentina,3.244633,1.481579,4.385965,14.017544,4.798246,0.300215,6,3,3.355669,0.107392,0.325773,1.006481,0.866667,0.495614,0.25100,1.868000,0.527,13.48,5.773443,0.492850,5.928502
1,CDMX,México,3.477388,1.388629,3.658863,12.963211,5.622074,0.178788,7,4,3.820318,0.050000,0.289320,0.614973,0.361111,0.421405,0.20200,1.910000,0.623,12.33,3.636191,0.397667,4.976668
2,Santiago,Chile,3.417211,1.462564,3.948718,13.764103,5.041026,0.311157,6,3,3.227544,0.100744,0.338889,1.051860,1.000000,0.430769,0.52500,1.890000,0.535,13.04,5.571009,0.480988,5.809878
3,Montevideo,Uruguay,3.227742,2.749495,7.747475,25.939394,4.479798,0.367266,6,3,3.081116,0.224523,0.360837,1.197588,1.000000,0.565657,0.17900,1.778000,0.356,13.40,8.471255,0.588490,6.884901
4,Madrid,España,3.518387,1.362343,3.790795,12.861925,6.079498,0.266684,8,5,3.783517,0.081917,0.294653,0.930159,1.000000,0.439331,0.18400,1.920000,0.606,12.13,4.256043,0.426240,5.262402
5,Sao Paulo,Brasil,3.430669,1.253704,3.370370,11.694444,6.726852,0.295414,8,4,4.324031,0.078571,0.266719,1.071208,0.222222,0.421296,0.23700,1.983000,0.682,10.38,3.258518,0.384944,4.849443
6,AGS,México,3.631461,1.797917,4.854167,17.020833,4.510417,0.328739,5,3,3.120614,0.140110,0.363556,1.182042,1.000000,0.593750,0.31400,1.868000,0.523,8.21,3.660575,0.461698,5.616984
7,GDL,México,3.573935,1.855615,5.069519,17.668449,4.705882,0.313413,6,3,3.208499,0.124392,0.342444,1.009314,1.000000,0.540107,0.28700,1.804000,0.511,10.81,4.845969,0.477331,5.773305
8,Pachuca,México,3.570259,1.563200,4.064000,13.952000,4.496000,0.214898,6,3,3.162323,0.095514,0.351108,0.852531,0.500000,0.384000,0.40100,1.860000,0.525,9.52,3.686997,0.439707,5.397074
9,Oaxaca,México,3.906913,1.883221,4.375839,17.369128,4.456376,0.234400,5,3,3.320697,0.107699,0.333850,1.157058,1.000000,0.516779,0.43500,1.846000,0.533,6.39,2.495120,0.399422,4.994219


In [32]:
#---------------------------
# Ranking DataFrame
#---------------------------

df_rankings = df_concat.copy() 
df_rankings = df_rankings[['Ciudad', 'País', 'Colab 0', 'Colab 10']]


#ranking of colaborativities between cities
for i in [0,10]:
    collab_i = list(df_concat['Colab '+str(i)])
    sorted_i = sorted(collab_i)[::-1]
    df_rankings['Ranking '+str(i)] = [sorted_i.index(v)+1 for v in collab_i]

#ranking between capitals
for i in [0,10]:
    collab_i_capitals = list(df_concat['Colab '+str(i)])[:6]
    sorted_i_capitals = sorted(collab_i_capitals)[::-1]
    L = [sorted_i_capitals.index(v)+1 for v in collab_i_capitals] + [None]*6
    df_rankings['Ranking Capitals '+str(i)] = L


df_rankings

,Ciudad,País,Colab 0,Colab 10,Ranking 0,Ranking 10,Ranking Capitals 0,Ranking Capitals 10
0,CABA,Argentina,5.773443,0.492850,4,4,2.0,2.0
1,CDMX,México,3.636191,0.397667,10,11,5.0,5.0
2,Santiago,Chile,5.571009,0.480988,5,5,3.0,3.0
3,Montevideo,Uruguay,8.471255,0.588490,3,3,1.0,1.0
4,Madrid,España,4.256043,0.426240,7,9,4.0,4.0
5,Sao Paulo,Brasil,3.258518,0.384944,11,12,6.0,6.0
6,AGS,México,3.660575,0.461698,9,7,NaN,NaN
7,GDL,México,4.845969,0.477331,6,6,NaN,NaN
8,Pachuca,México,3.686997,0.439707,8,8,NaN,NaN
9,Oaxaca,México,2.495120,0.399422,12,10,NaN,NaN


In [34]:
#df_concat.to_csv('Referenced_CSVs/Tidy_DataFrame.csv')
#df_rankings.to_csv('Referenced_CSVs/Rankings_0_10.csv')